In [30]:
import os
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torchinfo import summary
import time
import numpy as np
import pandas as pd

In [2]:
class SimpleMLP(nn.Module):
    def __init__(self, input_feature_dim, hidden_dim_1, hidden_dim_2, output_size=1): 
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(input_feature_dim, hidden_dim_1)
        self.relu1 = nn.ReLU()                        
        self.fc2 = nn.Linear(hidden_dim_1, hidden_dim_2)
        self.relu2 = nn.ReLU()
        self.output_logits = nn.Linear(hidden_dim_2, output_size)

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        logits = self.output_logits(x)
        
        # Scale sigmoid output to be between 0 and 5
        rating = 5.0 * torch.sigmoid(logits)
        return rating

In [15]:
INPUT_DIM = 422
H1, H2 = 256, 128

model_path = "./models/simple_mlp_ratings_statedict.pth"
device = torch.device("cpu")

model = SimpleMLP(INPUT_DIM, H1, H2).to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()
summary(model)

/tmp/ipykernel_18295/3220036478.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


Layer (type:depth-idx)                   Param #
SimpleMLP                                --
├─Linear: 1-1                            108,288
├─ReLU: 1-2                              --
├─Linear: 1-3                            32,896
├─ReLU: 1-4                              --
├─Linear: 1-5                            129
Total params: 141,313
Trainable params: 141,313
Non-trainable params: 0

### Model Size

In [16]:
model_size = os.path.getsize(model_path) 
print(f"Model Size on Disk: {model_size/ (1e6) :.2f} MB")

Model Size on Disk: 0.57 MB
